/////////////////////uncomment all pip installs///////////////////////////////////////////////


In [1]:
!pip install numpy faiss-cpu torch sentence-transformers transformers langchain


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: C:\Users\Janna\AppData\Local\Programs\Python\Python312\python.exe -m pip install --upgrade pip


In [2]:
!pip install fitz

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 528.1/528.1 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.3/43.3 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s eta 0:00:00


In [3]:
!pip uninstall -y fitz  # Remove any conflicting fitz packages /////////////////////uncomment///////////////////////////////////////////////
!pip install --upgrade pymupdf


Found existing installation: fitz 0.0.1.dev2
Uninstalling fitz-0.0.1.dev2:
  Successfully uninstalled fitz-0.0.1.dev2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 65.8 MB/s eta 0:00:00


In [4]:
import fitz
print(fitz.__version__)


1.25.5


In [5]:
import os
import re
import fitz  # PyMuPDF
import numpy as np
import faiss
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
# Step 1: Clone the GitHub repo /////////////////////uncomment///////////////////////////////////////////////
!git clone https://github.com/ammarahmed04eg/RAG-model-chatbot.git

# Step 2: Change directory to the folder with the PDFs
# %cd RAG-model-chatbot/PDF\ files
%cd RAG-model-chatbot


Cloning into 'RAG-model-chatbot'...
remote: Enumerating objects: 44, done.
remote: Counting objects: 100% (11/11), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 44 (delta 2), reused 2 (delta 0), pack-reused 33 (from 1)
Receiving objects: 100% (44/44), 61.97 MiB | 26.59 MiB/s, done.
Resolving deltas: 100% (5/5), done.
/content/RAG-model-chatbot


**Adjusting Fais ////////////////////////**

In [7]:
!pip install faiss-cpu


**Done Adjusting FAIS //////////////////**

In [8]:
# --- Configuration ---
# PDF_DIRECTORY = "/home/ubuntu/upload/PDF files/"

PDF_DIRECTORY = "PDF files/Digital/"
# FAISS_INDEX_PATH = "/home/ubuntu/faiss_index.idx"
FAISS_INDEX_PATH = "faiss_index.idx"

EMBEDDING_MODEL_NAME = "all-MiniLM-L6-v2"
# LLM_MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

In [9]:
from transformers import AutoTokenizer, AutoModelForCausalLM

LLM_MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model_name = LLM_MODEL_NAME

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code=True)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.29k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/551 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/608 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [10]:
# pdf_files = [f for f in os.listdir() if f.endswith('.pdf')]
# print("PDF files found:", pdf_files)
pdf_files = [f for f in os.listdir(PDF_DIRECTORY) if f.lower().endswith(".pdf")]
print("PDF files found:", pdf_files)

PDF files found: ['Digital Integrated Circuits - A Design Perspective.pdf']


In [11]:
# --- Text Processing Functions (from user's notebook) ---
def extract_text(pdf_path):
    text = ""
    try:
        doc = fitz.open(pdf_path)
        for page in doc:
            text += page.get_text("text") + "\n"
        doc.close()
    except Exception as e:
        print(f"Error extracting text from {pdf_path}: {e}")
        return ""
    return text.strip()

In [12]:
def clean_text(text):
    text = re.sub(r'\s+', ' ', text)
    # text = re.sub(r'[^a-zA-Z0-9.?!\s]', '', text)  # Keep spaces for sentence structure
    # text = re.sub(r'[^a-zA-Z0-9.,:;!?()\'\"\“\”-–—\s]', '', text)
    text = re.sub(r'[^a-zA-Z0-9.,:;!?()\'\"\“\\\”\\\–\\\—\s]', '', text)

    return text.lower().strip()

In [13]:
def chunk_text(text, chunk_size=1000, chunk_overlap=200):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", ". ", "? ", "! ", " ", ""]
    )
    return text_splitter.split_text(text)

Adjusted for a specific number of pdfs only

In [14]:
# --- Core RAG Pipeline Functions ---
def load_and_process_pdfs(pdf_dir, max_files=None):
    all_chunks = []
    if not os.path.exists(pdf_dir):
        print(f"Error: PDF directory not found at {pdf_dir}")
        print("Please ensure PDF files are uploaded to the correct location.")
        return [], []

    pdf_files = [f for f in os.listdir(pdf_dir) if f.endswith('.pdf')]
    if not pdf_files:
        print(f"No PDF files found in {pdf_dir}")
        return [], []

    # Limit the number of PDFs if max_files is specified
    if max_files is not None:
        pdf_files = pdf_files[:max_files]

    print(f"Found PDF files: {pdf_files}")
    processed_chunks_for_retrieval = []  # Store chunks for FAISS

    for pdf_file in pdf_files:
        pdf_path = os.path.join(pdf_dir, pdf_file)
        print(f"Processing {pdf_path}...")
        raw_text = extract_text(pdf_path)
        if not raw_text:
            continue
        cleaned_text = clean_text(raw_text)
        chunks = chunk_text(cleaned_text)
        processed_chunks_for_retrieval.extend(chunks)
        print(f"Extracted {len(chunks)} chunks from {pdf_file}.")

    print(f"Total chunks from all PDFs: {len(processed_chunks_for_retrieval)}")
    return processed_chunks_for_retrieval, processed_chunks_for_retrieval


In [15]:
def create_faiss_index(text_chunks, embedding_model_name, index_path):
    if not text_chunks:
        print("No text chunks available to create FAISS index.")
        return None, None
    print(f"Loading embedding model: {embedding_model_name}")
    try:
        embedding_model = SentenceTransformer(embedding_model_name)
        print("Embedding model loaded successfully.")
    except Exception as e:
        print(f"Error loading embedding model: {e}")
        return None, None

    print("Generating embeddings for text chunks...")
    embeddings = embedding_model.encode(text_chunks, show_progress_bar=True)
    print(f"Generated {len(embeddings)} embeddings with dimension {embeddings.shape[1]}.")

    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(np.array(embeddings).astype('float32'))

    print(f"Saving FAISS index to {index_path}...")
    faiss.write_index(index, index_path)
    print("FAISS index created and saved successfully.")
    return index, embedding_model

In [16]:
def load_llm_and_tokenizer(model_name):
    print(f"Attempting to load tokenizer for {model_name}...")
    try:
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        print(f"Tokenizer for {model_name} loaded successfully.")
    except Exception as e:
        print(f"Error loading tokenizer for {model_name}: {e}")
        return None, None

    print(f"Attempting to load model {model_name}...")
    try:
        if torch.cuda.is_available():
            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16,
                device_map="auto",
            )
            print(f"Model {model_name} loaded successfully on available CUDA device(s).")
        else:
            # model = AutoModelForCausalLM.from_pretrained(
            #     model_name,
            #     torch_dtype=torch.float16,
            # )
            # model = AutoModelForCausalLM.from_pretrained
            #  (
            #   model_name,
            #   torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            #   )

            model = AutoModelForCausalLM.from_pretrained(
                model_name,
                torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            )

            print(f"Model {model_name} loaded successfully on CPU.")
        return model, tokenizer
    except Exception as e:
        print(f"Error loading model {model_name}: {e}")
        return None, None

In [17]:
# def create_prompt_template(question, context):
# #     template = f"""You are a helpful assistant. Use the following context to answer the question accurately and concisely. If you don't know, say "I don't know."

# # Context:
# # {context}

# # Question: {question}

# # Answer:"""

#     # template = f"""You are a helpful assistant. Answer the question clearly and concisely.
#     # Use the provided context if it helps, but if it's missing, use your general knowledge.
#     # Do not include unrelated information.

#     # Context:
#     # {context}

#     # Question: {question}

#     # Answer:
#     # """



#     template = f"""[INST] You are an expert assistant. Given the context and question below, give a concise, clear answer. If context is incomplete, use your general knowledge. Always answer in 1-2 sentences.

#     Context:
#     {context}

#     Question: {question}


#     Answer: [/INST]
#     """

#     return template

In [18]:
import torch
from transformers import pipeline

# pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", trust_remote_code=True, device_map="auto")




Device set to use cuda:0


In [19]:
def create_prompt_template(question, context, tokenizer, max_tokens=2048):
    messages = [
        {
            "role": "system",
            "content": "You are an expert assistant. Given the context and question below, give a concise, clear answer. If context is incomplete, use your general knowledge. Always answer in 1-2 sentences.",
        },
        {"role": "user", "content": f"Context:\n{context}\n\nQuestion:\n{question}"},
    ]


    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


    # Truncate raw context before tokenizing if too long
    # prompt_tokens = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_tokens)
    # return prompt, prompt_tokens
    return prompt



In [20]:
def retrieve_context(query, faiss_index, embedding_model, text_chunks, top_k=3):
    if faiss_index is None or embedding_model is None:
        print("FAISS index or embedding model not available for retrieval.")
        return ""
    query_embedding = embedding_model.encode([query])
    distances, indices = faiss_index.search(np.array(query_embedding).astype('float32'), top_k)

    retrieved_chunks = []
    seen_texts = set()
    for idx in indices[0]:
        chunk = text_chunks[idx]
        if chunk not in seen_texts:
            retrieved_chunks.append(chunk)
            seen_texts.add(chunk)

    context = "\n\n".join(retrieved_chunks)
    print(f"Retrieved context (first 500 chars): {context[:500]}...")
    return context

In [21]:
def generate_answer(prompt, llm_model, llm_tokenizer, short=False):
    if llm_model is None or llm_tokenizer is None:
        return "Sorry, I am unable to generate an answer at this time."

    inputs = llm_tokenizer(prompt, return_tensors="pt", truncation=True, max_length=1024)
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    inputs = {k: v.to(device) for k, v in inputs.items()}

    output_sequences = llm_model.generate(
        **inputs,
        max_new_tokens=150,
        num_return_sequences=1,
        pad_token_id=llm_tokenizer.eos_token_id,
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        eos_token_id=llm_tokenizer.eos_token_id
    )
    full_generated_text = llm_tokenizer.decode(output_sequences[0], skip_special_tokens=True)

    # Extract answer after "Answer:"
    answer = ""
    if "Answer:" in full_generated_text:
        answer = full_generated_text.split("Answer:")[-1].strip()
    else:
        answer = full_generated_text.strip()

    # Shorten to 1-2 sentences
    if short:
        sentences = re.split(r'(?<=[.?!])\s+', answer)
        answer = ' '.join(sentences[:2]).strip()

    # outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
    # print(outputs[0]["generated_text"])

    return outputs

In [22]:
# # --- Main Execution ---
# if __name__ == "__main__":
print("Starting RAG pipeline setup...")

if not os.path.exists(PDF_DIRECTORY):
    print(f"PDF directory {PDF_DIRECTORY} does not exist. Creating it.")
    try:
        os.makedirs(PDF_DIRECTORY, exist_ok=True)
        print(f"Created directory {PDF_DIRECTORY}. Please add PDF files there to proceed fully.")
    except Exception as e:
        print(f"Could not create directory {PDF_DIRECTORY}: {e}")
        exit()

all_text_chunks, retrieval_text_chunks = load_and_process_pdfs(PDF_DIRECTORY, max_files=1)


Starting RAG pipeline setup...
Found PDF files: ['Digital Integrated Circuits - A Design Perspective.pdf']
Processing PDF files/Digital/Digital Integrated Circuits - A Design Perspective.pdf...
Extracted 1378 chunks from Digital Integrated Circuits - A Design Perspective.pdf.
Total chunks from all PDFs: 1378


In [23]:
faiss_index = None
embedding_model = None

if retrieval_text_chunks:
    faiss_index, embedding_model = create_faiss_index(retrieval_text_chunks, EMBEDDING_MODEL_NAME, FAISS_INDEX_PATH)
    if faiss_index and embedding_model:
        print("FAISS index and embedding model are ready.")
    else:
        print("Failed to create FAISS index or load embedding model.")
else:
    print("No text chunks processed. Skipping FAISS index creation.")
    print(f"Please ensure PDF files are available in {PDF_DIRECTORY}.")

llm_model, llm_tokenizer = load_llm_and_tokenizer(LLM_MODEL_NAME)
if not (llm_model and llm_tokenizer):
    print("Failed to load LLM or tokenizer. Exiting.")
    exit()
print("LLM and tokenizer are ready.")

print("RAG pipeline setup with prompt engineering is complete.")
print("You can now test the pipeline with questions.")


Loading embedding model: all-MiniLM-L6-v2


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding model loaded successfully.
Generating embeddings for text chunks...


Batches:   0%|          | 0/44 [00:00<?, ?it/s]

Generated 1378 embeddings with dimension 384.
Saving FAISS index to faiss_index.idx...
FAISS index created and saved successfully.
FAISS index and embedding model are ready.
Attempting to load tokenizer for TinyLlama/TinyLlama-1.1B-Chat-v1.0...
Tokenizer for TinyLlama/TinyLlama-1.1B-Chat-v1.0 loaded successfully.
Attempting to load model TinyLlama/TinyLlama-1.1B-Chat-v1.0...
Model TinyLlama/TinyLlama-1.1B-Chat-v1.0 loaded successfully on available CUDA device(s).
LLM and tokenizer are ready.
RAG pipeline setup with prompt engineering is complete.
You can now test the pipeline with questions.


In [24]:

# --- Test Section ---
test_questions = [
    # "ما هي عاصمة فرنسا؟",
    # "اشرح مفهوم الثقب الأسود ببساطة."
    # "Eleventh and Twelfth Dynasties are known as"
    "Who is the old man"
]

for test_question in test_questions:
    print(f"\n--- Testing with question: {test_question} ---")
    retrieved_ctx = ""
    if faiss_index and embedding_model and retrieval_text_chunks: # Only retrieve if index is built
        retrieved_ctx = retrieve_context(test_question, faiss_index, embedding_model, retrieval_text_chunks, top_k=10)

    if not retrieved_ctx:
        print("Could not retrieve context for the question or no PDFs processed. Answering without external context.")
        retrieved_ctx = "No relevant context"

    # final_prompt = create_prompt_template(test_question, retrieved_ctx)
    final_prompt = create_prompt_template(test_question, retrieved_ctx, llm_tokenizer, max_tokens=2048)
    print(f"Final prompt for LLM:\n{final_prompt}")
    # answer = generate_answer(final_prompt, llm_model, llm_tokenizer)
    # answer = generate_answer(final_prompt, llm_model, llm_tokenizer)

        # print("Testinggggggggggggggggg" , outputs[0]["generated_text"])

    # print(answer)

    # llm_model, llm_tokenizer
    answer = generate_answer(final_prompt, llm_model, llm_tokenizer, short=False)
    #  generate_answer(prompt, llm_model, llm_tokenizer, short=False)
    print(answer[0]["generated_text"])

print("\n--- All tests completed. ---")


--- Testing with question: Who is the old man ---
Retrieved context (first 500 chars): . the complexity of the manufac turing process delayed the full exploitation of these devices for two more decades. instead, 1 an intriguing overview of the evolution of digital integrated circuits can be found in murphy93. (most of the data in this overview has been extracted from this reference). it is accompanied by some of the his torically groundbreaking publications in the domain of digital ics. chapter1.fm page 11 friday, january 18, 2002 8:58 am 12 introduction chapter 1 the first practi...
Final prompt for LLM:
<|system|>
You are an expert assistant. Given the context and question below, give a concise, clear answer. If context is incomplete, use your general knowledge. Always answer in 1-2 sentences.</s>
<|user|>
Context:
. the complexity of the manufac turing process delayed the full exploitation of these devices for two more decades. instead, 1 an intriguing overview of the evolution of 

/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Token indices sequence length is longer than the specified maximum sequence length for this model (2285 > 2048). Running this sequence through the model will result in indexing errors
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (2048). Depending on the model, you may observe exceptions, performance degradation, or nothing at

<|system|>
You are an expert assistant. Given the context and question below, give a concise, clear answer. If context is incomplete, use your general knowledge. Always answer in 1-2 sentences.</s>
<|user|>
Context:
. the complexity of the manufac turing process delayed the full exploitation of these devices for two more decades. instead, 1 an intriguing overview of the evolution of digital integrated circuits can be found in murphy93. (most of the data in this overview has been extracted from this reference). it is accompanied by some of the his torically groundbreaking publications in the domain of digital ics. chapter1.fm page 11 friday, january 18, 2002 8:58 am 12 introduction chapter 1 the first practical mos integrated circuits were implemented in pmosonly logic and were used in applications such as calculators. the second age of the digital integrated cir cuit revolution was inaugurated with the introduction of the first microprocessors by intel in 1972 (the 4004) faggin72 and 1

In [28]:
import torch
from transformers import pipeline
import warnings

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")




test_questions = [
    "What a cmos?",
    "What is the difference between NMOS and PMOS?"
]


for question in test_questions:
    # question = "Who is the old man"

    warnings.filterwarnings("ignore", category=UserWarning, module="accelerate.big_modeling")

    context = ""
    if faiss_index and embedding_model and retrieval_text_chunks: # Only retrieve if index is built
        context = retrieve_context(question, faiss_index, embedding_model, retrieval_text_chunks, top_k=7)

    if not context:
        print("Could not retrieve context for the question or no PDFs processed. Answering without external context.")
        context = "No relevant context"





    # We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
    messages = [
            {
                "role": "system",
                "content": "You are an expert assistant. Given the context and question below, give a concise, clear answer. If context is incomplete, use your general knowledge. Always answer in 1-2 sentences.",
            },
            {"role": "user", "content": f"Context:\n{context}\n\nQuestion:\n{question}"},
        ]
    # warnings.filterwarnings("ignore", category=UserWarning, module="accelerate.big_modeling")

    prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

    # warnings.filterwarnings("ignore", category=UserWarning, module="accelerate.big_modeling")

    outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)

    warnings.filterwarnings("ignore", category=UserWarning, module="accelerate.big_modeling")

    print(outputs[0]["generated_text"])
print("Finished execution")

Device set to use cuda:0


Retrieved context (first 500 chars): . the cross section figure 2.1 cross section of an nwell cmos process. chapter2.fm page 42 friday, january 18, 2002 8:59 am section 2.2 manufacturing cmos integrated circuits 43 shown in figure 2.1 features an nwell cmos process, where the nmos transistors are implemented in the pdoped substrate, and the pmos devices are located in the nwell. modern processes are increasingly using a dualwell approach that uses both n and p wells, grown on top on a epitaxial layer, as shown in figure 2.2. we wil...
<|system|>
You are an expert assistant. Given the context and question below, give a concise, clear answer. If context is incomplete, use your general knowledge. Always answer in 1-2 sentences.</s>
<|user|>
Context:
. the cross section figure 2.1 cross section of an nwell cmos process. chapter2.fm page 42 friday, january 18, 2002 8:59 am section 2.2 manufacturing cmos integrated circuits 43 shown in figure 2.1 features an nwell cmos process, where the nmo

Working ..................

In [29]:
# import torch
# from transformers import pipeline

# pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0", torch_dtype=torch.bfloat16, device_map="auto")
# question = "Who is the old man"
# context = """  he brings our gear himself. he never wants anyone to carry anything.” “were different,” the old man said. “i let you carry things when you were five years old.” “i know it,” the boy said. “ill be right back. have another coffee. we have credit here.” he walked off, barefooted on the coral rocks, to the ice house where the baits were stored. the old man drank his coffee slowly. it was all he would have all day and he knew that he should take it. for a long time now eating had bored him and he never carried a lunch. he had a bottle of water in the bow of the skiff and that was all he needed for the day. the old man and the sea asiaing.com  10  the boy was back now with the sardines and the two baits wrapped in a newspaper and they went down the trail to the skiff, feeling the pebbled sand under their feet, and lifted the skiff and slid her into the water. 27 “good luck old man.” “good luck,” the old man said

# the old man and the sea asiaing.com  1  the old man and the sea by ernest hemingway www.asiaing.com to charlie shribner and to max perkins he was an old man who fished alone in a skiff in the gulf stream and he had gone eightyfour days now without taking a fish. in the first forty days a boy had been with him. but after forty days without a fish the boys parents had told him that the old man was now definitely and finally salao, which is the worst form of unlucky, and the boy had gone at their orders in another boat which caught three good fish the first week. it made the boy sad to see the old man come in each day with his skiff empty and he always went down to help him carry either the coiled lines or the gaff and harpoon and the sail that was furled around the mast. the sail was patched with flour sacks and, furled, it looked like the flag of permanent defeat. the old man was thin and gaunt with deep wrinkles in the back of his neck

# . they were strange shoulders, still powerful although very old, and the neck was still strong too and the creases did not show so much when the old man was asleep and his head fallen forward. his shirt had been patched so many times that it was like the sail and the patches were faded to many different shades by the sun. the 18 old mans head was very old though and with his eyes closed there was no life in his face. the newspaper lay across his knees and the weight of his arm held it there in the evening breeze. he was barefooted. the boy left him there and when he came back the old man was still asleep. “wake up old man,” the boy said and put his hand on one of the old mans knees. the old man opened his eyes and for a moment he was coming back from a long way away. then he smiled. “what have you got?” he asked. “supper,” said the boy. “were going to have supper.” the old man and the sea asiaing.com  6  “im not very hungry.” “come on and eat

# . “it is what a man must do.” they walked down the road to the old mans shack and all along the road, in the dark, barefoot men were moving, carrying the masts of their boats. when they reached the old mans shack the boy took the rolls of line in the basket and the harpoon and gaff and the old man carried the mast with the furled sail on his shoulder. “do you want coffee?” the boy asked. “well put the gear in the boat and then get some.” they had coffee from condensed milk cans at an early morning place that served fishermen. “how did you sleep old man?” the boy asked. he 26 was waking up now although it was still hard for him to leave his sleep. “very well, manolin,” the old man said. “i feel confident today.” “so do i,” the boy said. “now i must get your sardines and mine and your fresh baits. he brings our gear himself. he never wants anyone to carry anything.” “were different,” the old man said. “i let you carry things when you were five years old.” “i know it,” the boy said

# . do you think you can find an eightyfive?” “i can order one. “one sheet. thats two dollars and a half. who can we borrow that from?” “thats easy. i can always borrow two dollars and a half.” “i think perhaps i can too. but i try not to borrow. first you borrow. then you beg.” “keep warm old man,” the boy said. “remember we are in september.” “the month when the great fish come,” the old man said. “anyone can be a fisherman in may.” “i go now for the sardines,” the boy said. when the boy came back the old man was asleep in the chair and the sun was down. the boy took the old army blanket off the bed and spread it over the back of the chair and over the old mans shoulders. they were strange shoulders, still powerful although very old, and the neck was still strong too and the creases did not show so much when the old man was asleep and his head fallen forward

# . 27 “good luck old man.” “good luck,” the old man said. he fitted the rope lashings of the oars onto the thole pins and, leaning forward against the thrust of the blades in the water, he began to row out of the harbour in the dark. there were other boats from the other beaches going out to sea and the old man heard the dip and push of their oars even though he could not see them now the moon was below the hills. sometimes someone would speak in a boat. but most of the boats were silent except for the dip of the oars. they spread apart after they were out of the mouth of the harbour and each one headed for the part of the ocean where he hoped to find fish. the old man knew he was going far out and he left the smell of the land behind and rowed out into the clean early morning smell of the ocean

# . he had no mysticism about turtles although he had gone in turtle boats for many years. he was sorry for them all, even the great trunk backs that were as long as the skiff and weighed a ton. most people are heartless about turtles because a turtles heart will beat for hours after he has been cut up and butchered. but the old man thought, i have such a heart too and my feet and hands are like theirs. he ate the white eggs to give himself strength. he ate them all through may to be strong in september and october for the truly big fish. he also drank a cup of shark liver oil each day from the big drum in the shack where many of the fishermen kept their gear. it was there for all fishermen who wanted it. most fishermen hated the taste. but it was no worse than getting up at the hours that they rose and it was very good against all colds and grippes and it was good for the eyes. now the old man looked up and saw that the bird was circling again. “hes found fish,” he said aloud

# . up the road, in his shack, the old man was sleeping again. he was still sleeping on his face and the boy was sitting by him watching him. the old man was dreaming about the lions. the end. the old man and the sea asiaing.com 49 ernest hemingway, writer  born: 21 july 1899  birthplace: oak park, illinois  died: 2 july 1961 (suicide)  best known as: famously manly author of for whom the bell tolls hemingway is one of the 20th century's most famous american writers. his books include the sun also rises (1926), a farewell to arms (1929), and for whom the bell tolls (1940). hemingway's plainspoken nofrills writing style became so famous that it was (and is) frequently parodied. his dashing machismo was almost as famous as his writing: he lived in paris, cuba and key west, fancied bullfighting and big game hunting, and served as a war correspondent in wwii and the spanish civil war. he sealed his own notoriety when he killed himself with a shotgun in 1961



# """

# # We use the tokenizer's chat template to format each message - see https://huggingface.co/docs/transformers/main/en/chat_templating
# messages = [
#         {
#             "role": "system",
#             "content": "You are an expert assistant. Given the context and question below, give a concise, clear answer. If context is incomplete, use your general knowledge. Always answer in 1-2 sentences.",
#         },
#         {"role": "user", "content": f"Context:\n{context}\n\nQuestion:\n{question}"},
#     ]
# prompt = pipe.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
# outputs = pipe(prompt, max_new_tokens=256, do_sample=True, temperature=0.7, top_k=50, top_p=0.95)
# print(outputs[0]["generated_text"])